In [ ]:
import keras
from keras.layers import Dense, SimpleRNN, Input
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils import to_categorical

In [ ]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ

In [ ]:
import numpy as np

In [ ]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [ ]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [ ]:
data = tokenizer.texts_to_sequences([texts])

In [ ]:
print(data)

[[1, 4, 5, 6, 7, 8, 9, 2, 3, 10, 11, 12, 13, 2, 14, 15, 16, 17, 18, 19, 1, 20, 2, 3, 21, 22, 23, 24, 1, 25, 26]]


In [ ]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(31, 1000)


In [ ]:
inp_words = 3
n = res.shape[0]-inp_words

In [ ]:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [ ]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               144512    
                                                                 
 dense (Dense)               (None, 1000)              129000    
                                                                 
Total params: 273512 (1.04 MB)
Trainable params: 273512 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 6.9100 - accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 17ms/step - loss: 6.8842 - accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 18ms/step - loss: 6.8581 - accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 17ms/step - loss: 6.8315 - accuracy: 0.1429
Epoch 5/50
1/1 [==============================] - 0s 16ms/step - loss: 6.8041 - accuracy: 0.3571
Epoch 6/50
1/1 [==============================] - 0s 18ms/step - loss: 6.7757 - accuracy: 0.6786
Epoch 7/50
1/1 [==============================] - 0s 15ms/step - loss: 6.7460 - accuracy: 0.7857
Epoch 8/50
1/1 [==============================] - 0s 18ms/step - loss: 6.7146 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 17ms/step - loss: 6.6812 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 15ms/step - loss: 6.6455 - accuracy: 1.0000
Epoch 11/50
1/1 [==

In [ ]:
print(data)

[[1, 4, 5, 6, 7, 8, 9, 2, 3, 10, 11, 12, 13, 2, 14, 15, 16, 17, 18, 19, 1, 20, 2, 3, 21, 22, 23, 24, 1, 25, 26]]


In [ ]:
def buildPhrase(texts, str_len = 50):
  res = texts
  data1 = tokenizer.texts_to_sequences([texts])[0]

  for i in range(str_len):
    x = to_categorical(data1[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
    inp = x.reshape(1, inp_words, maxWordsCount)

    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data1.append(indx)

    res += " " + tokenizer.index_word[indx] # дописываем строку

  return res

 x = to_categorical(data1[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding

 как я понял в этот момент формируется пустой массив. Почему нет данных так и не смог разобраться. Гуглил в интернете, там тоже задаются таким вопросом.

In [27]:
res = buildPhrase("позитив добавляет годы")
print(res)

ValueError: cannot reshape array of size 0 into shape (1,3,1000)